In [14]:
import pandas as pd
import csv
import os
from tqdm import tqdm

In [15]:
directory = ".."  # Root directory


subdirectories = ["User-Revenue-Analytics/files/"]       

In [16]:
file_name = "installs.csv"
file_path = os.path.join(directory, *subdirectories, file_name)
df_installs = pd.read_csv(file_path,skipinitialspace=True)


file_name = "iap.csv"
file_path = os.path.join(directory, *subdirectories, file_name)
df_iap = pd.read_csv(file_path,skipinitialspace=True)

file_name = "rv.csv"
file_path = os.path.join(directory, *subdirectories, file_name)
df_rv = pd.read_csv(file_path,skipinitialspace=True)


In [17]:
df_inst_rv_merge = df_installs.merge(df_rv, how='left',on='user_id')
df_inst_iap_merge = df_installs.merge(df_iap, how='left',on='user_id')
df_merge = df_inst_rv_merge.merge(df_inst_iap_merge,how='outer',indicator=True)


In [18]:
def passed_houres(install_time, merge_time):
    time_difference = pd.Timestamp(merge_time) - pd.Timestamp(install_time)
    return (time_difference.total_seconds() / 3600)

In [19]:
df_merge.insert(2,column = "first_24hours_revenue", value =0) 
df_merge.insert(3,column = "end_of_day_revenue", value =0)

In [ ]:
def same_day(date1, date2):
    return date1.date() == date2.date()


def time_difference_hours(date1, date2):
    return (date2 - date1).total_seconds() / 3600


df_merge['install_time'] = pd.to_datetime(df_merge['install_time'])
df_merge['date'] = pd.to_datetime(df_merge['date'])


# this part may take 10-12 minutes
for index, row in tqdm(df_merge.iterrows(), total=len(df_merge)):
    install_time = row['install_time']
    date = row['date']
    time = row['time']
    purchase_value = row['purchase_value']
    ad_revenue = row['ad_revenue']
    
    if not pd.isna(time):
        if passed_houres(install_time, time) <= 24:
            df_merge.at[index, 'first_24hours_revenue'] = purchase_value
        if install_time.date() == pd.to_datetime(time).date():
            df_merge.at[index, 'end_of_day_revenue'] = purchase_value
            
    if not pd.isna(date):
        difference_hours = (install_time - date).total_seconds() / 3600
        if install_time.date() == date.date():
            df_merge.at[index, 'end_of_day_revenue'] = ad_revenue
            df_merge.at[index, 'first_24hours_revenue'] = ad_revenue

In [121]:
# with open('t.csv', 'w', newline='') as file:
#     writer = csv.writer(file)

In [122]:
# df_merge.to_csv('t.csv', sep=',', index=False, encoding='utf-8')

In [22]:
total_purchase = df_merge['purchase_value'].sum()
unique_users = df_installs['user_id'].nunique()
total_ad_revenue = df_rv['ad_revenue'].sum()
total_1day_revenue = df_merge['first_24hours_revenue'].sum()
total_end_of_1day_revenue = df_merge['end_of_day_revenue'].sum()
iap_arpu = total_purchase / unique_users
rv_arpu = total_ad_revenue / unique_users
overall_arpu =(total_purchase +total_ad_revenue)/ unique_users
end_of_1day_arpu = total_end_of_1day_revenue / unique_users
day1_ARPU = total_1day_revenue / unique_users

print('iap_arpu :', iap_arpu)
print('rv_arpu :', rv_arpu)
print('overall_arpu :', overall_arpu )
print('first_24hours_revenue_ARPU :', day1_ARPU)
print('end_of_1day_arpu :', end_of_1day_arpu)


iap_arpu : 0.7937312154068215
rv_arpu : 0.45701563835608755
overall_arpu : 1.2507468537629092
first_24hours_revenue_ARPU : 0.3430026700855399
end_of_1day_arpu : 0.2765293628219536
